# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [481]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [482]:
fires_dt.tail()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00
516,6,3,nov,tue,79.5,3.0,106.7,1.1,11.8,31,4.5,0.0,0.00


# Get X and Y

Create the features data frame and target data.

In [483]:
X = fires_dt.drop(columns='area')
Y = fires_dt['area']

In [484]:
import random
import sklearn
from sklearn.model_selection import train_test_split, cross_validate

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [485]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import cross_validate 
from sklearn.ensemble import GradientBoostingRegressor

num_cols = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
cat_cols = ['month', 'day']


In [486]:
preproc1 = ColumnTransformer([('standard', StandardScaler(), num_cols),
                                ('one_hot_1', OneHotEncoder(handle_unknown='ignore'), cat_cols)])
preproc1

ColumnTransformer(transformers=[('standard', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('one_hot_1',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [487]:
preproc2 = ColumnTransformer([('robust', RobustScaler(), num_cols),
                            ('one_hot_2', OneHotEncoder(handle_unknown='ignore'), cat_cols)])
preproc2

ColumnTransformer(transformers=[('robust', RobustScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('one_hot_2',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [488]:
# Pipeline A = preproc1 + baseline
pipe_A = Pipeline([('preprocessing', preproc1),
                      ('linear', LinearRegression())])

pipe_A
                     

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standard', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('one_hot_1',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('linear', LinearRegression())])

In [489]:
pipe_A.fit(X_train, Y_train )

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standard', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('one_hot_1',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('linear', LinearRegression())])

In [490]:
# Pipeline B = preproc2 + baseline
pipe_B = Pipeline([('preprocess2', preproc2),
                   ('ridge', Ridge())])

pipe_B

Pipeline(steps=[('preprocess2',
                 ColumnTransformer(transformers=[('robust', RobustScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('one_hot_2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('ridge', Ridge())])

In [ ]:
# Pipeline C = preproc1 + advanced model

pipe_C = Pipeline([('preprocess1', preproc1),
                   ('decision_tree', DecisionTreeRegressor())])

pipe_C

Pipeline(steps=[('preprocess1',
                 ColumnTransformer(transformers=[('standard', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('one_hot_1',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('decision_tree', DecisionTreeRegressor())])

In [492]:
# Pipeline D = preproc2 + advanced model

pipe_D = Pipeline([('preproces2', preproc2),
                   ('boosting', GradientBoostingRegressor()) ])

pipe_D    

Pipeline(steps=[('preproces2',
                 ColumnTransformer(transformers=[('robust', RobustScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('one_hot_2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('boosting', GradientBoostingRegressor())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [493]:
from sklearn.model_selection import GridSearchCV

In [502]:
scoring = ['r2', 'neg_mean_squared_error', 'f1', 'accuracy', 'precision', 'recall']
#scoring = ['neg_mean_squared_error']

param_grid_A = {'linear__fit_intercept': [True, False],
                'linear__copy_X': [True,False],
                'linear__positive': [True, False]  
                }

grid_cv_A = GridSearchCV( estimator=pipe_A, 
                           param_grid=param_grid_A,
                           scoring = scoring,
                           cv=5,
                           refit='neg_mean_squared_error')
grid_cv_A.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('standard',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('one_hot_1',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('linear', LinearRegression())]),
             param_grid={'linear__copy_X': [True, False],
                         'linear__fit_intercept': [True, False],
                         'linear__positive': [True, False]},
             refit='neg_mean_squared_error',
             scoring=['r2', 'neg_mean_squared_error', 'f1', 'accuracy',
                      'precision', 'recall'])

In [504]:
print('Best parameters: ', grid_cv_A.best_params_)
print('Best negative mean squared error score: ', grid_cv_A.best_score_)

Best parameters:  {'linear__copy_X': True, 'linear__fit_intercept': False, 'linear__positive': True}
Best negative mean squared error score:  -2450.080969543677


In [505]:
param_grid_B = {'ridge__alpha': [1,5,10],
                'ridge__fit_intercept': [True,False]
                }

grid_cv_B = GridSearchCV( estimator=pipe_B, 
                           param_grid=param_grid_B,
                           scoring = scoring,
                           cv=5,
                           refit='neg_mean_squared_error')
grid_cv_B.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess2',
                                        ColumnTransformer(transformers=[('robust',
                                                                         RobustScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('one_hot_2',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': [1, 5, 10],
                         'ridge__fit_intercept': [True, False]},
             refit='neg_mean_squared_error',
             scoring=['r2', 'neg_mean_squared_error', 'f1', 'accuracy',
                      'precision', 'recall'])

In [506]:
print('Best parameters for model B: ', grid_cv_B.best_params_)
print('Best negative mean squared error score for model B: ', grid_cv_B.best_score_)

Best parameters for model B:  {'ridge__alpha': 10, 'ridge__fit_intercept': False}
Best negative mean squared error score for model B:  -2415.149916472387


In [507]:

param_grid_C = {'decision_tree__criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                'decision_tree__splitter': ['best','random'],
                
                }

grid_cv_C = GridSearchCV( estimator=pipe_C, 
                           param_grid=param_grid_C,
                           scoring = scoring,
                           cv=5,
                           refit='neg_mean_squared_error')
grid_cv_C.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess1',
                                        ColumnTransformer(transformers=[('standard',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('one_hot_1',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('decision_tree',
                                        DecisionTreeRegressor())]),
             param_grid={'decision_tree__criterion': ['squared_error',
                                                      'friedman_mse',
                                                      'absolute_error',
                                                      'poisson'],
                         'decision_tree__splitter': ['best', 'random']},
             refit='neg_mean_squared_error',
             scoring=['r2', 'neg_mean_squared_error', 'f1', 'accuracy',
                      'precision', 'recall'])

In [508]:
print('Best parameters for model C: ', grid_cv_C.best_params_)
print('Best negative mean squared error score for model C: ', grid_cv_C.best_score_)

Best parameters for model C:  {'decision_tree__criterion': 'poisson', 'decision_tree__splitter': 'random'}
Best negative mean squared error score for model C:  -3697.355180159409


In [509]:
param_grid_D = {'boosting__loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
                'boosting__n_estimators': [50, 75, 100],
                'boosting__criterion': ['friedman_mse', 'squared_error']  
                }

grid_cv_D = GridSearchCV( estimator=pipe_D, 
                           param_grid=param_grid_D,
                           scoring = scoring,
                           cv=5,
                           refit='neg_mean_squared_error')
grid_cv_D.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preproces2',
                                        ColumnTransformer(transformers=[('robust',
                                                                         RobustScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('one_hot_2',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('boosting',
                                        GradientBoostingRegressor())]),
             param_grid={'boosting__criterion': ['friedman_mse',
                                                 'squared_error'],
                         'boosting__loss': ['squared_error', 'absolute_error',
                                            'huber', 'quantile'],
                         'boosting__n_estimators': [50, 75, 100]},
             refit='neg_mean_squared_error',
             scoring=['r2', 'neg_mean_squared_error', 'f1', 'accuracy',
                      'precision', 'recall'])

In [510]:
print('Best parameters for model D: ', grid_cv_D.best_params_)
print('Best negative mean squared error score for model D: ', grid_cv_D.best_score_)

Best parameters for model D:  {'boosting__criterion': 'squared_error', 'boosting__loss': 'absolute_error', 'boosting__n_estimators': 100}
Best negative mean squared error score for model D:  -2412.3961565343125


# Evaluate

+ Which model has the best performance?

In [511]:
neg_mean_squared_errors = {'A': grid_cv_A.best_score_, 'B': grid_cv_B.best_score_, 'C':grid_cv_C.best_score_, 
                           'D': grid_cv_D.best_score_}
print(dict(sorted(neg_mean_squared_errors.items(),key=lambda item :item[1], reverse=True)))

{'D': -2412.3961565343125, 'B': -2415.149916472387, 'A': -2450.080969543677, 'C': -3697.355180159409}


# Export

+ Save the best performing model to a pickle file.

In [ ]:

import pickle
# The best model is pipe_D
# We add the optimized parameters we found using GridSearchCV to pipe_D
# and save this revised model to a pickle file

best_model= Pipeline([('preproces2', preproc2),
                   ('boosting', GradientBoostingRegressor(criterion='squared_error', 
                        loss='absolute_error', 
                        n_estimators= 100)) ])

file_path = '../../05_src/best_model.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model, file)


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.